In [1]:
import math, copy
import multiprocessing as mp

In [2]:
size = range(2, 50)

In [3]:
# Part 1: Imagine a circle of elves, numbered from 1, each holding a gift.
# Going around the circle in order, each elf steals a gift from the elf on
# their left. Any elf without a gift is removed from the circle. Which elf
# ends up with all the gifts?

# Solution: First I solved this problem by walking the list, which was
# prohibitively slow for large n
def part1a(x):
    p1 = {}
    elves = list(range(x))
    while len(elves) > 1:
        for i, e in enumerate(elves):
            elves.pop((i+1) % len(elves))
    p1[x] = elves[0] + 1
    return p1

In [4]:
# I tried speeding up execution with threading or multiprocessing, but
# concluded that the computation is CPU-bound and modifies a single massive
# data structure - I would have to chunk it manually. Along the way, I ran
# the above code for n from 2 to 50. Noticing a pattern in the output, I
# derived a formula for both results.
    
# part1: 2^n => 1, then follow 2^n odd numbers (so the last is 2^n+1 - 1 => 2^n+1 - 1)

def part1b(x):
    powers = [2**n for n in range(25)]
    odds = [1 + 2*n for n in range(x)]
    
    t = [a for a in powers if a <= x]
    r = x - t[-1]
    return {x: odds[r]}

In [5]:
with mp.Pool() as pool:
    a = pool.map(part1a, size)
    b = pool.map(part1b, size)
a == b

True

In [6]:
%%time
part1b(3001330)

CPU times: user 475 ms, sys: 70.6 ms, total: 546 ms
Wall time: 577 ms


{3001330: 1808357}

In [7]:
# Part 2: Imagine instead that the elves steal gifts from the elf directly
# across the circle from them, or the elf on the left (from the stealer's
# view) if there are two options. Now which elf ends up with all the gifts?

# Solution: Again, this code was technically correct but incredibly slow; it would've run for 24+ hrs on the problem input.

def part2a(x):
    p2 = {}
    elves = list(range(x))
    while len(elves) > 1:
        ref = copy.deepcopy(elves)
        for r in ref:
            if r in elves:
                j = elves.index(r)
                elves.pop((j + math.floor(len(elves) / 2)) % len(elves))
    p2[x] = elves[0] + 1
    
    return p2

In [8]:
# part2: 1 => 1, if n => n then start 1, 2, 3 until n => n/2, then continue sequence of odds until n => n

# This still takes a while to run
def part2b(x):
    p2 = {}
    odds = [1 + 2*n for n in range(x)]
    numbers = odds
    for n in range(1, x + 1):
        o = numbers[0]
        if n == x:
            p2[n] = o

        numbers.pop(0)
        if n == o:
            numbers = list(range(1, n + 1))
        if n/2 == o:
            numbers = odds

    return p2

In [9]:
def part2c(x):
    odds = [1 + 2*n for n in range(x)]
    
    for r in range(15):
        if 3**r >= x:
            a = r - 1
            break
    
    p = 3**a
    r = x - p
    d = 3**(a+1) - p
    
    if r <= d//2:
        y = r
    else:
        s = r - d//2
        i = odds.index(p)
        y = odds[i + s]

    return {x: y}

In [10]:
with mp.Pool() as pool:
    a = pool.map(part2a, size)
    b = pool.map(part2b, size)
    c = pool.map(part2c, size)

a == b and b == c

True

In [11]:
%%time
# part2a(300) -> 3ms
# part2a(3000) -> 166ms
# part2a(30000) -> 15s
# part2a(300000) -> 18min

CPU times: user 16min 13s, sys: 4.5 s, total: 16min 17s
Wall time: 17min 42s


{300000: 122853}

In [12]:
%%time
part2b(3001330)

CPU times: user 42min 24s, sys: 12.2 s, total: 42min 36s
Wall time: 1h 1min


{3001330: 1407007}

In [43]:
%%time
part2c(3001330)

CPU times: user 521 ms, sys: 77.8 ms, total: 599 ms
Wall time: 1.09 s


{3001330: 1407007}